# Data Cleaning and Preprocessing

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

pd.options.mode.chained_assignment = None

Load and drop unnecessary attribute, e.g., `Filming_Location`

In [20]:
dataset_path = "../dataset/"
data = pd.read_csv(dataset_path + "data_joined.csv")
data = data.drop('Filming_Location', axis=1)
a = data
a.reset_index(drop=True, inplace=True)
a.head()

,Movie_Title,Movie_ID,Budget,Cast,Crew,Studios,Genre,Keywords,Languages,Countries,Release_Data,Runtime,Gross_worldwide,Rating,Rating_Count,ListOfCertificate
0,Jumanji: Welcome to the Jungle,2283362,"$90,000,000","Dwayne Johnson,Karen Gillan,Kevin Hart,Jack Bl...","Scott Rosenberg,Erik Sommers,Chris McKenna,Jak...","Columbia Pictures,Matt Tolmach Productions,Rad...","Action,Adventure,Comedy","jungle,adventurer,trapped in a game,action her...",English,United States,2017-12-20,1 hour 59 minutes,"$995,339,117",6.9,389K,PG-13
1,Transformers: Revenge of the Fallen,1055369,"$200,000,000","Shia LaBeouf,Megan Fox,Josh Duhamel,Tyrese Gib...","Ehren Kruger,Michael Bay,Roberto Orci,Alex Kur...","Dreamworks Pictures,Paramount Pictures,Tom DeS...","Action,Adventure,Sci-Fi","optimus prime character,megatron character,toa...","English,Spanish",United States,2009-06-24,2 hours 29 minutes,"$836,303,693",5.9,412K,PG-13
2,Star Wars: Episode VII - The Force Awakens,2488496,"$245,000,000","Daisy Ridley,John Boyega,Oscar Isaac,Domhnall ...","J.J. Abrams,Michael Arndt,Lawrence Kasdan","Lucasfilm,Bad Robot,Walt Disney Pictures","Action,Adventure,Sci-Fi","crash landing,reboot,actor reprises previous r...",English,United States,2015-12-18,2 hours 18 minutes,"$2,071,310,218",7.8,939K,PG-13
3,Spider-Man,145487,"$139,000,000","Tobey Maguire,Kirsten Dunst,Willem Dafoe,James...","Sam Raimi,David Koepp,Stan Lee,Steve Ditko","Columbia Pictures,Marvel Enterprises,Laura Zis...","Action,Adventure,Sci-Fi","superhero,peter parker character,ben parker ch...",English,United States,2002-05-03,2 hours 1 minute,"$825,025,036",7.4,827K,PG-13
4,Iron Man Three,1300854,"$200,000,000","Robert Downey Jr.,Guy Pearce,Gwyneth Paltrow,D...","Drew Pearce,Stan Lee,Shane Black","Marvel Studios,Paramount Pictures,DMG Entertai...","Action,Adventure,Sci-Fi","terrorist,billionaire,human bomb,armor,rescue",English,United States,2013-05-03,2 hours 10 minutes,"$1,215,577,205",7.1,860K,PG-13


## Missing Values

Check for missing values

In [21]:
a.isnull().sum()

Movie_Title             0
Movie_ID                0
Budget               2749
Cast                    7
Crew                    3
Studios                43
Genre                   1
Keywords               24
Languages              10
Countries               5
Release_Data            0
Runtime                 6
Gross_worldwide      1032
Rating                  8
Rating_Count            8
ListOfCertificate    1191
dtype: int64

Drop all records missing `Gross_worldwide`

In [22]:
listToDrop = list(a[a['Gross_worldwide'].isna()].index)
a = a.drop(listToDrop)

Fill missing values:

| Attribute           | Fill with |
|---------------------|-----------|
| `Budget`            | $0        |
| `Runtime`           | 0         |
| `Rating`            | 5.0       |
| `Rating_Count`      | 0K        |
| `ListOfCertificate` | G         |

Other null attributes are filled with empty string.

In [23]:
a['Budget'] = a['Budget'].fillna("$0")
a['Runtime'] = a['Runtime'].fillna("0")
a['Rating'] = a['Rating'].fillna(5.0)
a['Rating_Count'] = a['Rating_Count'].fillna("0K")
a['ListOfCertificate'] = a['ListOfCertificate'].fillna("G")
a = a.fillna("")

Drop all records with empty attributes

In [24]:
a = a.reset_index(drop=True)

## Currency Attributes

**Input**: Input dataframe with unprocessed currency

**Output**: Processed one

Currency symbols in `Budget` and `Gross_worldwide`

In [25]:
currency = [str(s)[0] for s in a['Budget'].unique()]
print(set(currency))
currency = [str(s)[0] for s in a['Gross_worldwide'].unique()]
print(set(currency))

{'$', '€'}
{'$'}


Eliminate `$`,`€` and `,` in money string and convert the string to integer

In [26]:
def parse_currency(before_parsed):
    if '$' in before_parsed:
        after_parsed = int(before_parsed.strip('$').replace(',', ""))
    else:
        after_parsed = int(int(before_parsed.strip('€').replace(',', "")) * 1.14)
    return after_parsed

In [27]:
a['Budget'] = a['Budget'].apply(parse_currency)
a['Gross_worldwide'] = a['Gross_worldwide'].apply(parse_currency)
a.head()

,Movie_Title,Movie_ID,Budget,Cast,Crew,Studios,Genre,Keywords,Languages,Countries,Release_Data,Runtime,Gross_worldwide,Rating,Rating_Count,ListOfCertificate
0,Jumanji: Welcome to the Jungle,2283362,90000000,"Dwayne Johnson,Karen Gillan,Kevin Hart,Jack Bl...","Scott Rosenberg,Erik Sommers,Chris McKenna,Jak...","Columbia Pictures,Matt Tolmach Productions,Rad...","Action,Adventure,Comedy","jungle,adventurer,trapped in a game,action her...",English,United States,2017-12-20,1 hour 59 minutes,995339117,6.9,389K,PG-13
1,Transformers: Revenge of the Fallen,1055369,200000000,"Shia LaBeouf,Megan Fox,Josh Duhamel,Tyrese Gib...","Ehren Kruger,Michael Bay,Roberto Orci,Alex Kur...","Dreamworks Pictures,Paramount Pictures,Tom DeS...","Action,Adventure,Sci-Fi","optimus prime character,megatron character,toa...","English,Spanish",United States,2009-06-24,2 hours 29 minutes,836303693,5.9,412K,PG-13
2,Star Wars: Episode VII - The Force Awakens,2488496,245000000,"Daisy Ridley,John Boyega,Oscar Isaac,Domhnall ...","J.J. Abrams,Michael Arndt,Lawrence Kasdan","Lucasfilm,Bad Robot,Walt Disney Pictures","Action,Adventure,Sci-Fi","crash landing,reboot,actor reprises previous r...",English,United States,2015-12-18,2 hours 18 minutes,2071310218,7.8,939K,PG-13
3,Spider-Man,145487,139000000,"Tobey Maguire,Kirsten Dunst,Willem Dafoe,James...","Sam Raimi,David Koepp,Stan Lee,Steve Ditko","Columbia Pictures,Marvel Enterprises,Laura Zis...","Action,Adventure,Sci-Fi","superhero,peter parker character,ben parker ch...",English,United States,2002-05-03,2 hours 1 minute,825025036,7.4,827K,PG-13
4,Iron Man Three,1300854,200000000,"Robert Downey Jr.,Guy Pearce,Gwyneth Paltrow,D...","Drew Pearce,Stan Lee,Shane Black","Marvel Studios,Paramount Pictures,DMG Entertai...","Action,Adventure,Sci-Fi","terrorist,billionaire,human bomb,armor,rescue",English,United States,2013-05-03,2 hours 10 minutes,1215577205,7.1,860K,PG-13


## Multivalued Attributes

Multivalued data is seperated by comma `,`

`Cast`, `Genre`, `Studios`, `ListOfCertificate`, `Keywords`, `Languages`, `Countries` are multivalued so we convert them to `List` data structure

In [28]:
def parse_multi_value(field_value):
    array_value = field_value.split(',')
    if '' in array_value:
        return []
    return array_value

In [29]:
cols = ['Cast', 'Genre', 'Studios', 'ListOfCertificate', 'Keywords', 'Languages', 'Countries', 'Crew']
for col in cols:
    a[col] = a[col].apply(parse_multi_value)

Note!
After change to list we will need this function this to load again in another part

In [30]:
"""

import ast
cols =['Cast', 'Genre', 'Studios', 'ListOfCertificate','Keywords', 'Languages', 'Countries']
for col in cols:
    a[col]=a[col].apply(ast.literal_eval)
    
"""

"\n\nimport ast\ncols =['Cast', 'Genre', 'Studios', 'ListOfCertificate','Keywords', 'Languages', 'Countries']\nfor col in cols:\n    a[col]=a[col].apply(ast.literal_eval)\n    \n"

Process wrong values in `ListOfCertificate`

In [31]:
def get_unique_certificates(dataframe):
    certificates = set()
    for i in dataframe['ListOfCertificate']:
        for y in i:
            certificates.add(y)
    return certificates

In [32]:
get_unique_certificates(a)

{'G', 'GP', 'M', 'M/PG', 'NC-17', 'PG', 'PG-13', 'R', 'X'}

Some of those certificates are out of date and does not follow MPAA's newest policy. They should be replaced:

- M, GP and M/PG replaced by PG

- X replaced by NC-17

In [33]:
def update_certificates(certificates):
    new_certificates = set()
    type1 = ['M', 'GP', 'M/PG']
    type2 = ['X']
    for i in certificates:
        if i in type1:
            new_certificates.add('PG')
        elif i in type2:
            new_certificates.add('NC-17')
        else:
            new_certificates.add(i)
    return list(new_certificates)

In [34]:
a['ListOfCertificate'] = a['ListOfCertificate'].apply(update_certificates)
get_unique_certificates(a)

{'G', 'NC-17', 'PG', 'PG-13', 'R'}

## Multivalued attributes with gross and count

1. Input
- `dataframe`: Dataframe need to be process
- `col_name`: Name of the column to process, `Cast`, `Genre` and `Studios` for example

2. Output
- Output: Dataframe contains information related to the columns combined with Money and Count

In [35]:
def parseWithMoneyAndCount(dataframe, col_name):
    result = []
    count = []
    gross = []
    for i, record in enumerate(dataframe[col_name]):
        for x in record:
            # Save results to corresponding array
            result.append(x)
            gross.append(dataframe['Gross_worldwide'][i])
            count.append(1)
    # Make dataframe
    t = pd.DataFrame({col_name: result, 'Money': gross, 'Count': count})
    # Remove duplicates and sum corresponding columns
    result = t.groupby(col_name).sum()
    # Rearrange dataframe
    sort_by_money = result.sort_values('Money', ascending=False)
    return sort_by_money

# Normalize time data
### 1.  Runtime data in hours-minutes
Function to convert hours to minutes

In [36]:
def convertTime(time):
    time = str(time)
    time = time.replace(" ", "")
    if "hour" in time:
        hours = 0
        if "hours" in time:
            hours = int(time[0]) * 60
            after_eliminate_hour = time.replace("hours", "")
        else:
            hours = 60
            after_eliminate_hour = time.replace("hour", "")
        minutes = 0
        if "minute" in after_eliminate_hour:
            if "minutes" in after_eliminate_hour:
                minutes = int(after_eliminate_hour.replace("minutes", "")[1:])
            else:
                minutes = 1
        return int(hours) + int(minutes)
    else:
        return int(time.replace("minutes", ""))

# Normalize time data for the dataset

In [37]:
a['Runtime'] = a['Runtime'].apply(convertTime)

In [38]:
a['Runtime'].value_counts()

100    255
97     253
95     238
90     236
96     235
      ... 
271      1
357      1
185      1
207      1
194      1
Name: Runtime, Length: 158, dtype: int64

### 2. Release_date in Day, Month & Year
Split Day, Month & Year in each Movie

In [39]:
a['Release_Data'] = pd.to_datetime(a['Release_Data'], format='%Y-%m-%d')
a['Release_Year'] = a['Release_Data'].apply(lambda x: x.year)
a['Release_Month'] = a['Release_Data'].apply(lambda x: x.month)
a['Release_Day'] = a['Release_Data'].apply(lambda x: x.day)

## Process Rating và Rating_Count
Convert rating & rating_count to float data type

In [40]:
def convertRatingCount(rating_count):
    rate = str(rating_count)
    if 'M' in rate:
        return int(float(rate.replace("M", "")) * 1000000)
    elif 'K' in rate:
        return int(float(rate.replace("K", "")) * 1000)
    else:
        return int(rate)

In [41]:
a['Rating_Count'] = a['Rating_Count'].apply(convertRatingCount)
a['Rating'] = a['Rating'].astype(float)

## Process attributes with 0 value

Fill 0 values:

| Attribute           | Fill with |
|---------------------|-----------|
| `Budget`            | Min       |
| `Runtime`           | Mean      |

In [42]:
## Find Budget 0
indexNZero = list(a[a['Budget'] != 0].index)
indexZero = list(a[a['Budget'] == 0].index)
min = np.min(list(a.iloc[indexNZero]['Budget']))
a['Budget'] = a['Budget'].apply(lambda x: min if x == 0 else x)
## Find Runtime 0
indexNZero = list(a[a['Runtime'] != 0].index)
indexZero = list(a[a['Runtime'] == 0].index)
mean = np.mean(list(a.iloc[indexNZero]['Runtime']))
a['Runtime'] = a['Runtime'].apply(lambda x: mean if x == 0 else x)

## Xuất ra file CSV

In [44]:
a.to_csv("../dataset/processed/cleaned_data.csv", index=False)